# 📊 Oracle Unifier Data Pull

This notebook connects to Oracle Unifier Cloud and pulls data from the `i-project_tasks` business process.

**Your URL:** `https://uae1.unifier.oraclecloud.com/aldar/bp/route/1/i-project_tasks?__uref=uuu556219734`

---

## 1️⃣ Install Required Packages

In [ ]:
# Install required packages
!pip install requests pandas openpyxl -q
print("✓ Packages installed successfully!")

## 2️⃣ Import Libraries

In [ ]:
import requests
import json
import pandas as pd
from getpass import getpass
from datetime import datetime
import urllib3
from google.colab import files

# Disable SSL warnings (for testing only)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

print("✓ Libraries imported successfully!")

## 3️⃣ Configure Your Credentials

Enter your Oracle Unifier credentials below. Your password will be hidden when you type it.

In [ ]:
# ============================================
# CONFIGURATION - UPDATE THESE VALUES
# ============================================

# Your Oracle Unifier base URL
BASE_URL = "https://uae1.unifier.oraclecloud.com"

# Your company/tenant name (from URL: .../aldar/...)
TENANT = "aldar"

# The business process name
BP_NAME = "i-project_tasks"

# The unique reference from your URL
UREF = "uuu556219734"

# Enter your credentials (password will be hidden)
USERNAME = input("Enter your Oracle Unifier username: ")
PASSWORD = getpass("Enter your Oracle Unifier password: ")

print("\n✓ Configuration complete!")

## 4️⃣ Oracle Unifier Client Class

In [ ]:
class OracleUnifierClient:
    """
    Client for connecting to Oracle Unifier Cloud.
    Supports both Basic Auth and REST API authentication.
    """
    
    def __init__(self, base_url, tenant, username, password):
        self.base_url = base_url.rstrip('/')
        self.tenant = tenant
        self.username = username
        self.password = password
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
            'Accept': 'application/json',
            'Content-Type': 'application/json'
        })
        self.authenticated = False
        
    def authenticate(self):
        """
        Authenticate with Oracle Unifier using Basic Auth.
        """
        print("🔐 Authenticating...")
        
        # Set up basic authentication
        self.session.auth = (self.username, self.password)
        
        # Test with a simple API call
        test_url = f"{self.base_url}/ws/rest/service/v1/admin/company"
        
        try:
            response = self.session.get(test_url, timeout=30)
            
            if response.status_code == 200:
                self.authenticated = True
                print("✅ Authentication successful!")
                return True
            elif response.status_code == 401:
                print("❌ Authentication failed: Invalid credentials")
                return False
            else:
                print(f"⚠️ Unexpected response: {response.status_code}")
                # Try alternative authentication
                return self._try_alternative_auth()
                
        except requests.exceptions.RequestException as e:
            print(f"❌ Connection error: {str(e)}")
            return False
    
    def _try_alternative_auth(self):
        """
        Try alternative authentication methods.
        """
        print("   Trying alternative authentication...")
        
        # Try with encoded credentials in header
        import base64
        credentials = base64.b64encode(f"{self.username}:{self.password}".encode()).decode()
        self.session.headers['Authorization'] = f'Basic {credentials}'
        
        # Remove session auth to use header instead
        self.session.auth = None
        
        test_url = f"{self.base_url}/ws/rest/service/v1/admin/company"
        response = self.session.get(test_url, timeout=30)
        
        if response.status_code == 200:
            self.authenticated = True
            print("✅ Alternative authentication successful!")
            return True
        
        return False
    
    def get_bp_records(self, bp_name, project_number=None, limit=100, offset=0):
        """
        Retrieve business process records using the REST API.
        
        Args:
            bp_name: Business process name (e.g., 'i-project_tasks')
            project_number: Optional project number filter
            limit: Maximum records to return
            offset: Pagination offset
            
        Returns:
            dict: API response with records
        """
        if not self.authenticated:
            print("❌ Please authenticate first!")
            return None
            
        # REST API endpoint for BP records
        endpoint = f"{self.base_url}/ws/rest/service/v1/bp/records"
        
        params = {
            'bp_name': bp_name,
            'limit': limit,
            'offset': offset
        }
        
        if project_number:
            params['project_number'] = project_number
        
        print(f"📥 Fetching records from {bp_name}...")
        
        try:
            response = self.session.get(endpoint, params=params, timeout=60)
            
            if response.status_code == 200:
                data = response.json()
                print(f"✅ Successfully retrieved data!")
                return data
            else:
                print(f"❌ Error {response.status_code}: {response.text[:200]}")
                return None
                
        except Exception as e:
            print(f"❌ Error: {str(e)}")
            return None
    
    def get_all_records(self, bp_name, project_number=None, batch_size=100):
        """
        Retrieve ALL records with pagination.
        
        Args:
            bp_name: Business process name
            project_number: Optional project filter
            batch_size: Records per batch
            
        Returns:
            list: All records combined
        """
        all_records = []
        offset = 0
        
        while True:
            print(f"   Fetching batch starting at offset {offset}...")
            
            data = self.get_bp_records(
                bp_name=bp_name,
                project_number=project_number,
                limit=batch_size,
                offset=offset
            )
            
            if not data:
                break
            
            # Extract records from response
            records = data.get('data', data.get('records', []))
            
            if not records:
                break
                
            all_records.extend(records)
            
            # Check if we got fewer records than requested (last batch)
            if len(records) < batch_size:
                break
                
            offset += batch_size
        
        print(f"\n✅ Total records retrieved: {len(all_records)}")
        return all_records

print("✓ Client class defined!")

## 5️⃣ Connect and Authenticate

In [ ]:
# Create client instance
client = OracleUnifierClient(
    base_url=BASE_URL,
    tenant=TENANT,
    username=USERNAME,
    password=PASSWORD
)

# Authenticate
success = client.authenticate()

if not success:
    print("\n⚠️ If authentication keeps failing, you may need to:")
    print("   1. Check your username and password")
    print("   2. Verify your account has API access")
    print("   3. Contact your Oracle Unifier administrator")

## 6️⃣ Fetch Project Tasks Data

In [ ]:
# Fetch records from the i-project_tasks business process
data = client.get_bp_records(
    bp_name=BP_NAME,
    limit=100  # Adjust as needed
)

# Display raw response structure
if data:
    print("\n📋 Response structure:")
    print(json.dumps(data, indent=2)[:2000] + "..." if len(json.dumps(data)) > 2000 else json.dumps(data, indent=2))

## 7️⃣ Convert to DataFrame

In [ ]:
def convert_to_dataframe(data):
    """
    Convert API response to pandas DataFrame.
    """
    if not data:
        print("❌ No data to convert!")
        return pd.DataFrame()
    
    # Handle different response formats
    if 'data' in data:
        records = data['data']
    elif 'records' in data:
        records = data['records']
    elif isinstance(data, list):
        records = data
    else:
        records = [data]
    
    df = pd.DataFrame(records)
    print(f"\n✅ Created DataFrame with {len(df)} rows and {len(df.columns)} columns")
    return df

# Convert to DataFrame
df = convert_to_dataframe(data)

# Display info
if not df.empty:
    print("\n📊 Column names:")
    for col in df.columns:
        print(f"   - {col}")
    
    print("\n📋 Preview (first 5 rows):")
    display(df.head())

## 8️⃣ Export to Excel/CSV

In [ ]:
# Generate timestamp for filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Export to Excel
excel_filename = f"unifier_project_tasks_{timestamp}.xlsx"
df.to_excel(excel_filename, index=False)
print(f"✅ Saved to {excel_filename}")

# Export to CSV
csv_filename = f"unifier_project_tasks_{timestamp}.csv"
df.to_csv(csv_filename, index=False)
print(f"✅ Saved to {csv_filename}")

# Download files
print("\n📥 Downloading files...")
files.download(excel_filename)
files.download(csv_filename)

## 9️⃣ Fetch ALL Records (with Pagination)

In [ ]:
# Uncomment and run this cell to fetch ALL records
# This handles pagination automatically

# all_records = client.get_all_records(
#     bp_name=BP_NAME,
#     batch_size=100
# )

# if all_records:
#     df_all = pd.DataFrame(all_records)
#     print(f"\n✅ Total records: {len(df_all)}")
#     display(df_all.head())
#     
#     # Export all records
#     all_filename = f"unifier_all_tasks_{timestamp}.xlsx"
#     df_all.to_excel(all_filename, index=False)
#     files.download(all_filename)

---

## 🔧 Troubleshooting

### Authentication Issues
- **401 Unauthorized**: Check your username and password
- **403 Forbidden**: Your account may not have API access
- **Connection timeout**: Check your network/VPN connection

### API Access
Oracle Unifier REST API requires:
1. Valid Unifier user credentials
2. Appropriate permissions to access BP data
3. REST API enabled for your tenant

### Contact Support
If issues persist, contact your Oracle Unifier administrator to:
- Verify your API access permissions
- Check if REST API is enabled for your tenant
- Get the correct API endpoints for your configuration